<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/1-boolean-bow-tfidf-search-systems/1_Buscador_Booleano_bag_of_words_e_buscador_com_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Usar o BM25 implementado pelo pyserini para buscar queries no TREC-DL 2020
Documentação referencia: https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md
2. Implementar um buscador booleano/bag-of-words.
3. Implementar um buscador com TF-IDF
4. Avaliar implementações 1, 2, e 3 no TREC-DL 2020 e calcular o nDCG@10
Nos itens 2 e 3:
Fazer uma implementação que suporta buscar eficientemente milhões de documentos.
Não se pode usar bibliotecas como sklearn, que já implementam o BoW e TF-IDF.

## Installing dependencies and preparing the environment

The following chunks of code are from [Pyserini development install guide](https://github.com/castorini/pyserini/blob/master/docs/installation.md#development-installation), not exactly in the same order as it is presented there. Special thanks to Marcus Borela and Monique Louise, for sharing their code and insightful discussions on the WhatsApp group.

In [1]:
# install conda for Google Colab (thanks Monique!)
# running this cell will restart your notebook's kernel
!pip install condacolab -q

import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


The next two code cells check how much RAM memory you're using, as well as if you are connected to GPU or not. Note that you probably should use those configurations for a better experience with this notebook.

In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [3]:
# install Pyserini dev dependencies
!conda install -c conda-forge maven

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [21]:
!pip install torch -q
!pip install pyserini -q
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 59.9 MB/s eta 0:00:00


In [5]:
# In order to perform some of the steps, it is needed to mount your Google Drive
# You will need to authorize the connection in the prompt 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
# This cell creates a folder (if it doesn't exist already) for this project in 
# your Google Drive
main_path = '/content/drive/MyDrive/week1-notebook/'

import os

if not os.path.exists(main_path):
  os.makedirs(main_path)
else:
  print('Path already exists')

<!-- In order to be able do perform `git clone` from Google Colab, you should follow those steps to authorize t -->

In [7]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules

Cloning into 'pyserini'...
remote: Enumerating objects: 7242, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 7242 (delta 175), reused 194 (delta 112), pack-reused 6954
Receiving objects: 100% (7242/7242), 4.28 MiB | 21.48 MiB/s, done.
Resolving deltas: 100% (5220/5220), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 20.92 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '7b84f773225b5973b4533dfa0aa18653409a6146'


In [8]:
!pip install -e pyserini -q

  Preparing metadata (setup.py) ... done


In [9]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Clone and build Anserini package
As pointed by [Pyserini development installation](https://github.com/castorini/pyserini/blob/master/docs/installation.md#development-installation), the next step is to clone and build Anserini following the [documentation steps](https://github.com/castorini/anserini#-getting-started).

In [11]:
!git clone https://github.com/castorini/anserini.git --recurse-submodules

fatal: destination path 'anserini' already exists and is not an empty directory.


In [12]:
# Thanks Monique, once again, for figuring that out
!mvn -f anserini/pom.xml clean package appassembler:assemble

[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.pom (12 kB at 32 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/33/maven-plugins-33.pom (11 kB at 534 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/33/maven-parent-33.pom (44 kB at 1.2 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/21/apache-21.pom (17 kB at 1.2 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.jar (62 kB at 2.7 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/mojo/appassembler-maven-plugin/2.1.0/appassembler-maven-plugin-2.1.0.pom (19 kB at 808 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/code

In [19]:
!cd pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

## Get dataset
Following the instructions from the [reference docs](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md) provided to solve the problem, let's download the data.

In [14]:
# creates a new working directory
!mkdir collections/msmarco-passage

# downloads the data to the newly created directory 
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage

# extracts data
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

mkdir: cannot create directory ‘collections/msmarco-passage’: No such file or directory
--2023-03-09 01:49:50--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  13.2MB/s    in 2m 28s  

2023-03-09 01:52:18 (6.82 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


Next, the data needs to be converted to JSONL files, which can be easily achieved through the provided functions.

In [15]:
!python pyserini/tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path collections/msmarco-passage/collection.tsv \
 --output-folder collections/msmarco-passage/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

In [16]:
!cp anserini/target/anserini-0.20.1-SNAPSHOT-fatjar.jar pyserini/pyserini/resources/jars

### Indexing data in Lucene

In [17]:
# This takes several minutes to complete
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/msmarco-passage/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-03-09 01:53:29,714 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-03-09 01:53:29,716 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-03-09 01:53:29,716 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-03-09 01:53:29,716 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: collections/msmarco-passage/collection_jsonl
2023-03-09 01:53:29,716 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-03-09 01:53:29,716 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-03-09 01:53:29,717 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-03-09 01:53:29,717 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-03-09 01:53:29,717 INFO  [main] 

## Pyserini's BM25 Baseline

In [18]:
topics = 'dl20'
qrels_eval = 'dl20-passage'

Pyserini's documentation mention that `k1` and `b` were tuned with grid search for MS MARCO Passage dataset. Here, I chose to use the default values that are mentioned in [Practical BM25 - Part 3: Considerations for Picking b and k1 in Elasticsearch](https://www.elastic.co/blog/practical-bm25-part-3-considerations-for-picking-b-and-k1-in-elasticsearch), a blogpost.

`hits` refers to the number of documents to retrieve for each query. Here, I set it to 25.

In [22]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {topics} \
  --output runs/run.{topics}.bm25tuned.txt \
  --output-format msmarco \
  --hits 25 \
  --bm25 --k1 1.20 --b 0.75

Setting BM25 parameters: k1=1.2, b=0.75
Running dl20 topics, saving to runs/run.dl20.bm25tuned.txt...
100% 200/200 [00:08<00:00, 24.81it/s]


To use the official TREC evaluation tool to compute metrics, we need to converd the files into TREC format.

In [23]:
# Still following Pyserini docs.
!python -m pyserini.eval.convert_msmarco_run_to_trec_run \
   --input runs/run.{topics}.bm25tuned.txt \
   --output runs/run.{topics}.bm25tuned.trec

!python pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input pyserini/tools/topics-and-qrels/qrels.{qrels_eval}.txt \
   --output collections/msmarco-passage/qrels.{qrels_eval}.trec

Done!
Done!


In [24]:
!pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   collections/msmarco-passage/qrels.{qrels_eval}.trec runs/run.{topics}.bm25tuned.trec

map                   	all	0.2211
ndcg_cut_10           	all	0.4836


Cool! That was achieved using the tutorial provided by the professors. But maybe we should check if the same result is achieved when using the Python interface. The following code is based on [this demo](https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb). Since we have a pre-built index, it should be simple.

In [25]:
!pip install pyserini -q

In [26]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('./indexes/lucene-index-msmarco-passage')

ModuleNotFoundError: ignored

## Boolean